In [109]:
import numpy as np
import pandas as pd
import requests

pd.options.display.max_colwidth = 1000

In [118]:
hostname = "http://lookup-service-prod.mlb.com"
path = "/json/named.transaction_all.bam?sport_code='mlb'&start_date={0}&end_date={1}".format("20080101","20081231")

In [119]:
r = requests.get('{0}{1}'.format(hostname, path))
j = r.json()

In [120]:
df = pd.DataFrame(j['transaction_all']['queryResults']['row'])

In [121]:
df.to_csv("./2008_injury_mlb_api.csv")

In [113]:
placed = df[(df.note.str.contains("placed")) & (df.note.str.contains("disabled list"))]
placed2 = df[(df.note.str.contains("placed")) & (df.note.str.contains("injured list"))]

print(len(placed))
print(len(placed2))

2
0


In [107]:
row = dict()
df3 = pd.DataFrame(columns = ['playerid','injury_start_date','injury_end_date'])

for k,v in placed.iterrows():
    row['playerid'] = v.player_id
    row['injury_start_date'] = v.trans_date
    
    # resolve this players time injured
    activated = df[(df.player_id == v.player_id) & 
                   (df.trans_date > v.trans_date) & 
                   ((df.note.str.contains("activated")) & ((df.note.str.contains("disabled list")) | (df.note.str.contains("injured list"))))]
    optioned = df[(df.player_id == v.player_id) & 
                  (df.trans_date > v.trans_date) & 
                  (df.type.isin(['Optioned','Released','Signed as Free Agent',
                                 'Declared Free Agency','Outrighted','Returned',
                                 'Claimed Off Waivers','Retired']))]
    combined = activated.append(other=optioned)
    combined = combined.sort_values(by=['trans_date'])
    
    if len(combined) > 0:
        row['injury_end_date'] = combined.iloc[0].trans_date
    else:
        activated = df[(df.player_id == v.player_id) & 
                   (df.trans_date > v.trans_date) & 
                   (df.note.str.contains("activated"))]
        combined = activated.sort_values(by=['trans_date'])
        if len(combined) > 0:
            row['injury_end_date'] = combined.iloc[0].trans_date
        else:
            activated = df[(df.player_id == v.player_id) & 
                   (df.trans_date > v.trans_date) & 
                   (df.note.str.contains("roster status changed by"))]
            combined = activated.sort_values(by=['trans_date'])
            if len(combined) > 0:
                row['injury_end_date'] = combined.iloc[0].trans_date
            
    tmp = pd.Series(row)
    df3 = df3.append(other=tmp, ignore_index=True)
    row = dict()
    
df3.head()

,playerid,injury_start_date,injury_end_date
0,489265,2009-03-27T00:00:00,2009-05-30T00:00:00
1,430161,2009-03-27T00:00:00,2009-11-03T00:00:00
2,433585,2009-03-27T00:00:00,2009-11-13T00:00:00
3,150061,2009-03-27T00:00:00,2009-04-27T00:00:00
4,407113,2009-03-27T00:00:00,2009-12-30T00:00:00


In [108]:
df3[df3.injury_end_date.isnull()]

,playerid,injury_start_date,injury_end_date
35,430657,2009-04-05T00:00:00,NaN
107,429718,2009-04-26T00:00:00,NaN
178,434565,2009-05-24T00:00:00,NaN
246,114260,2009-06-19T00:00:00,NaN
381,429665,2009-08-24T00:00:00,NaN


In [87]:
# df3.to_csv("./2008_injury_mlb_api.csv")

In [ ]:
df[df.player_id == '453214'].loc[:,['type','trans_date','resolution_date','note']]

In [143]:
# fill missing end date as Nov 1st (end of regular season or thereabouts)
for i in range(2009,2020):
    inj = pd.read_csv('./{0}_injury_mlb_api.csv'.format(i))
    
    inj.injury_end_date = inj.injury_end_date.fillna('{0}-11-01T00:00:00'.format(i))
    
    inj.to_csv('./{0}_injury_mlb_api.csv'.format(i))

In [154]:
# append all the injury datasets into one
cols = ['playerid', 'injury_start_date', 'injury_end_date']
all_inj = pd.DataFrame(columns=cols)

all_inj.head()

for i in range(2009,2020):
    inj = pd.read_csv('./{0}_injury_mlb_api.csv'.format(i))
    
    all_inj = pd.concat([all_inj, inj])
    break

all_inj.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1'], axis=1, inplace=True)
all_inj.head()

,playerid,injury_start_date,injury_end_date
0,489265,2009-03-27T00:00:00,2009-05-30T00:00:00
1,430161,2009-03-27T00:00:00,2009-11-03T00:00:00
2,433585,2009-03-27T00:00:00,2009-11-13T00:00:00
3,150061,2009-03-27T00:00:00,2009-04-27T00:00:00
4,407113,2009-03-27T00:00:00,2009-12-30T00:00:00


In [164]:
all_inj['days_injured'] = pd.to_datetime(all_inj.injury_end_date) - pd.to_datetime(all_inj.injury_start_date)
all_inj.days_injured = all_inj.days_injured / np.timedelta64(1, 'D')
print(all_inj.days_injured.dtype)
all_inj.head()

float64


,playerid,injury_start_date,injury_end_date,days_injured
0,489265,2009-03-27T00:00:00,2009-05-30T00:00:00,64.0
1,430161,2009-03-27T00:00:00,2009-11-03T00:00:00,221.0
2,433585,2009-03-27T00:00:00,2009-11-13T00:00:00,231.0
3,150061,2009-03-27T00:00:00,2009-04-27T00:00:00,31.0
4,407113,2009-03-27T00:00:00,2009-12-30T00:00:00,278.0
